# 🤖 Entraînement RL Agent sur Google Colab

Ce notebook est **spécifiquement pour entraîner l'agent RL** sur Colab avec GPU.
Le modèle entraîné sera téléchargé pour utilisation locale.


## 📦 Étape 1 : Installation des dépendances


In [ ]:
# Installation PyTorch avec CUDA 11.8 (pour GPU Colab)
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 -q

# Dépendances principales pour RL
%pip install diffusers transformers accelerate -q
%pip install stable-baselines3[extra] gymnasium -q
%pip install pillow numpy requests pydantic pydantic-settings -q

print("✅ Dépendances installées")


## 🔧 Étape 2 : Cloner et configurer le projet


In [ ]:
# Remplacer par l'URL de votre repository GitHub
REPO_URL = "https://github.com/votre-user/votre-repo.git"

import os

# Cloner le repo
if not os.path.exists("Projet_fil_rouge_ML_DL"):
    !git clone {REPO_URL}

%cd Projet_fil_rouge_ML_DL

# Configuration pour Colab (GPU) - Entraînement RL
os.environ["SD_DEVICE"] = "cuda"
os.environ["SD_DTYPE"] = "float16"
os.environ["OUTPUT_DIR"] = "outputs"
os.environ["MODELS_DIR"] = "models"
os.environ["RL_AGENT_PATH"] = "models/rl_agent.zip"

print("✅ Projet configuré pour entraînement RL")


## 🔍 Étape 3 : Vérifier le GPU


In [ ]:
!nvidia-smi

import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n⚠️ IMPORTANT: Assurez-vous d'avoir un GPU activé pour l'entraînement RL")
else:
    print("\n❌ ERREUR: Aucun GPU détecté. Activez le GPU dans Runtime > Change runtime type")


## 🚀 Étape 4 : Entraîner l'agent RL

**⚠️ IMPORTANT :**
- Entraînement 10k steps : ~1-2 heures sur GPU
- Entraînement 20k steps : ~2-4 heures sur GPU
- Le modèle sera sauvegardé dans `models/rl_agent.zip`


In [ ]:
# Configuration de l'entraînement
TOTAL_TIMESTEPS = 10000  # Ajustez selon vos besoins (10000 recommandé minimum)
SAVE_PATH = "models/rl_agent.zip"

print(f"🚀 Démarrage entraînement RL agent...")
print(f"📊 Steps d'entraînement: {TOTAL_TIMESTEPS}")
print(f"⏱️ Temps estimé: {TOTAL_TIMESTEPS // 5000:.1f}-{TOTAL_TIMESTEPS // 2500:.1f} heures")
print("\n" + "="*50 + "\n")

# Importer les modules
from app.models.rl_agent import RLOptimizer
from training.rl_env import PromptOptimizationEnv

# Créer environnement et agent
env = PromptOptimizationEnv()
agent = RLOptimizer(env=env)

# Entraîner l'agent
agent.train(
    total_timesteps=TOTAL_TIMESTEPS,
    save_path=SAVE_PATH
)

print("\n" + "="*50)
print("✅ Entraînement terminé !")
print(f"📦 Modèle sauvegardé: {SAVE_PATH}")


## ✅ Étape 5 : Vérifier le modèle entraîné


In [ ]:
# Vérifier que le modèle existe et a la bonne taille
import os
model_path = "models/rl_agent.zip"

if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"✅ Modèle trouvé: {model_path}")
    print(f"📦 Taille: {size_mb:.2f} MB")
    
    # Lister tous les fichiers du modèle
    import zipfile
    with zipfile.ZipFile(model_path, 'r') as zip_ref:
        files = zip_ref.namelist()
        print(f"📁 Fichiers dans le modèle: {len(files)}")
        for f in files[:5]:  # Afficher les 5 premiers
            print(f"   - {f}")
        if len(files) > 5:
            print(f"   ... et {len(files) - 5} autres fichiers")
else:
    print(f"❌ ERREUR: Modèle non trouvé à {model_path}")


## 💾 Étape 6 : Sauvegarder dans Google Drive


In [ ]:
# Monter Google Drive pour sauvegarder le modèle
from google.colab import drive
drive.mount('/content/drive')

# Créer dossier de sauvegarde
DRIVE_MODELS_DIR = "/content/drive/MyDrive/ai-creative-studio/models"
os.makedirs(DRIVE_MODELS_DIR, exist_ok=True)

# Copier le modèle vers Drive
model_name = "rl_agent.zip"
drive_model_path = f"{DRIVE_MODELS_DIR}/{model_name}"

!cp models/rl_agent.zip {drive_model_path}

print(f"✅ Modèle sauvegardé dans Google Drive")
print(f"📦 Chemin: {drive_model_path}")
print(f"\n💡 Vous pourrez télécharger depuis Drive ou utiliser sur Colab")


## 📥 Étape 7 : Télécharger le modèle localement

**Option 1 : Téléchargement direct depuis Colab**


In [ ]:
# Télécharger le modèle directement depuis Colab
from google.colab import files

model_path = "models/rl_agent.zip"
if os.path.exists(model_path):
    print(f"📥 Téléchargement de {model_path}...")
    files.download(model_path)
    print("✅ Téléchargement terminé !")
    print("\n💡 Placez le fichier dans le dossier 'models/' de votre projet local")
else:
    print(f"❌ Modèle non trouvé: {model_path}")


**Option 2 : Télécharger depuis Google Drive**

1. Allez sur [Google Drive](https://drive.google.com)
2. Naviguez vers `ai-creative-studio/models/`
3. Téléchargez `rl_agent.zip`
4. Placez-le dans `models/rl_agent.zip` de votre projet local


## 🧪 Étape 8 : Test rapide du modèle (optionnel)

Tester que le modèle fonctionne correctement avant téléchargement.


In [ ]:
# Test rapide d'optimisation de prompt
test_prompt = "a cat"

print(f"🧪 Test d'optimisation pour: '{test_prompt}'...")

try:
    result = agent.optimize_prompt(
        base_prompt=test_prompt,
        n_iterations=5  # Test rapide avec 5 itérations
    )
    
    print("\n📊 Résultats:")
    print(f"  Prompt original: {result['original_prompt']}")
    print(f"  Prompt optimisé: {result['optimized_prompt']}")
    print(f"  Score original: {result['original_score']:.2f}")
    print(f"  Score optimisé: {result['optimized_score']:.2f}")
    print(f"  Amélioration: {result['improvement']:+.2f}")
    print(f"  Paramètres: {result['best_params']}")
    print("\n✅ Modèle fonctionne correctement !")
except Exception as e:
    print(f"⚠️ Erreur lors du test: {e}")
    print("💡 Le modèle peut quand même fonctionner localement")
